In [1]:
import pandas as pd
import numpy as np

#### County: Unemployment, Median Household Income, Poverty

#### State: Infrastructure, homelessness

In [2]:
xl = pd.ExcelFile('C:/Users/User/Documents/GitHub/AnotherTown/Data/Health/2020 County Health Rankings Data.xlsx')

In [3]:
sh1 = pd.read_excel(xl, sheet_name=xl.sheet_names[-4], skiprows=[0], 
                    usecols=['FIPS','State','County','% Unemployed'])
nan = sh1.iloc[0,2]
sh1=sh1.dropna(axis=0, subset=['County'])
sh1['% Unemployed'] = pd.qcut(sh1.iloc[:,-1], 10, labels=[i for i in range(1,11)], )
sh1.head(3)

,FIPS,State,County,% Unemployed
1,1001,Alabama,Autauga,5
2,1003,Alabama,Baldwin,5
3,1005,Alabama,Barbour,9


In [4]:
sh2 = pd.read_excel(xl, sheet_name= xl.sheet_names[-2], skiprows=[0], 
                    usecols=['FIPS','State','County','Median Household Income'])
sh2 = sh2.dropna(axis=0, subset=['County'])
sh2['Median Household Income'] = pd.qcut(-1*sh2.iloc[:,-1], 10, labels=[i for i in range(1,11)], )
sh2.head(3)

,FIPS,State,County,Median Household Income
1,1001,Alabama,Autauga,3
2,1003,Alabama,Baldwin,3
3,1005,Alabama,Barbour,10


In [5]:
sh1 = sh1.merge(sh2, on=['State', 'FIPS', 'County'])
sh1.head(3)

,FIPS,State,County,% Unemployed,Median Household Income
0,1001,Alabama,Autauga,5,3
1,1003,Alabama,Baldwin,5,3
2,1005,Alabama,Barbour,9,10


In [26]:
poverty = pd.read_csv('../Data/Economy/poverty_est18all.csv', skiprows=[0,1,2,4],
                     usecols = ['State FIPS Code', 
                                'County FIPS Code', 
                                'Poverty Percent, All Ages'])
poverty['FIPS'] = [row[0] * 1000 + row[1] for i, row in poverty.iterrows()]
poverty = poverty.drop(axis=1, columns=['County FIPS Code', 'State FIPS Code'])
poverty['Poverty Percent, All Ages'] = pd.to_numeric(poverty['Poverty Percent, All Ages'],errors='coerce')
poverty['Poverty Percent, All Ages'] = pd.qcut(poverty.iloc[:,-2], 10, labels=[i for i in range(1,11)], )
poverty.head(3)

,"Poverty Percent, All Ages",FIPS
0,7,1000
1,5,1001
2,2,1003


In [7]:
infra = pd.read_csv('../Data/Economy/infrastructure.csv')
infra.head(3)

,State,Roads in poor condition,Deficient bridges,Dams at high risk,State highway speding per driver
0,50. Florida,Roads in poor condition: 1.3% (2nd lowest),Deficient bridges: 2.1% (3rd lowest),Dams at high hazard risk: 6.3% (9th lowest),State highway spending per driver: $457 (22nd ...
1,49. Georgia,Roads in poor condition: 1.9% (5th lowest),Deficient bridges: 4.7% (7th lowest),Dams at high hazard risk: 11.0% (15th lowest),State highway spending per driver: $254 (3rd l...
2,48. Minnesota,Roads in poor condition: 4.1% (20th lowest),Deficient bridges: 5.3% (11th lowest),Dams at high hazard risk: 5.1% (5th lowest),State highway spending per driver: $555 (18th ...


In [8]:
infra['State'] = [i.split('. ')[1] for i in infra['State']]
infra['Roads in poor condition'] = [float(i.split(': ')[1].split('%')[0]) for i in infra['Roads in poor condition']]
infra['Deficient bridges'] = [float(i.split(': ')[1].split('%')[0]) for i in infra['Deficient bridges']]
infra['Dams at high risk'] = [float(i.split(': ')[1].split('%')[0]) for i in infra['Dams at high risk']]
infra['State highway speding per driver'] = [-1* int(i.split('$')[1].split(' ')[0].replace(',','')) for i in infra['State highway speding per driver']]
infra = infra.set_index('State')
infra.head(3)

,Roads in poor condition,Deficient bridges,Dams at high risk,State highway speding per driver
State,,,,
Florida,1.3,2.1,6.3,-457
Georgia,1.9,4.7,11.0,-254
Minnesota,4.1,5.3,5.1,-555


In [9]:
for c, col in enumerate(infra.columns):
    infra[col] =  pd.qcut(infra[col], 10, labels=[i for i in range(1,11)], )
infra.head(3)

,Roads in poor condition,Deficient bridges,Dams at high risk,State highway speding per driver
State,,,,
Florida,1,1,2,6
Georgia,1,2,3,10
Minnesota,4,3,1,4


In [10]:
infra['met_avg'] = [(row[0] * 5 + row[1] * 4 + row[2] * 1 + row[3] * 4) / 14
 for c, row in infra.iterrows()]
infra.head(3)

,Roads in poor condition,Deficient bridges,Dams at high risk,State highway speding per driver,met_avg
State,,,,,
Florida,1,1,2,6,2.5
Georgia,1,2,3,10,4.0
Minnesota,4,3,1,4,3.5


In [11]:
homeless = pd.read_csv('../Data/Economy/homeless_reg_map.csv', index_col = 'State Name', usecols=['State Name', 'Rate Per 10000'])
homeless['Rate Per 10000'] =  pd.qcut(homeless['Rate Per 10000'], 10, labels=[i for i in range(1,10)], duplicates='drop', )
homeless.head(3)

,Rate Per 10000
State Name,
Alaska,8
Alabama,1
Arkansas,3


In [12]:
comb = infra.join(homeless)
comb['state_total'] = comb.iloc[:,-2:].mean(axis=1)
comb.head(3)

,Roads in poor condition,Deficient bridges,Dams at high risk,State highway speding per driver,met_avg,Rate Per 10000,state_total
State,,,,,,,
Florida,1,1,2,6,2.5,6,4.25
Georgia,1,2,3,10,4.0,4,4.00
Minnesota,4,3,1,4,3.5,7,5.25


In [29]:
df = sh1.copy(deep=True)
df = df.merge(comb.iloc[:,-1],on='State')
df = df.merge(poverty, how='left', on='FIPS')
df.head(3)

,FIPS,State,County,% Unemployed,Median Household Income,state_total,"Poverty Percent, All Ages"
0,1001,Alabama,Autauga,5,3,2.642857,5
1,1003,Alabama,Baldwin,5,3,2.642857,2
2,1005,Alabama,Barbour,9,10,2.642857,10


In [32]:
df.to_csv('../Data/Cleaned/Economy_Raw.csv', index=False)